In [73]:
import pandas as pd
import copy
import numba
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 252 ms


In [2]:
game_data = pd.read_excel('/Users/jason.katz/Downloads/Analytics_Attachment.xlsx', "2016_17_NBA_Scores")
team_data = pd.read_excel('/Users/jason.katz/Downloads/Analytics_Attachment.xlsx', "Division_Info")

time: 158 ms


In [146]:
def create_game_dicts(game_data):
    games = []
    for index, row in game_data.iterrows():
        game = {}
        if row['Winner'] == 'Home':
            game[row['Home Team']] = 'Winner'
            game[row['Away Team']] = 'Loser'
        else:
            game[row['Home Team']] = 'Loser'
            game[row['Away Team']] = 'Winner'
        games.append(game)
    return games

time: 3.88 ms


In [147]:
def simulate_games(games, teams):
    for game in games:
        for team in game:
            teams[team]['Games_Played'] += 1
            if game[team] == 'Loser':
                teams[team]['Max_Wins'] -= 1

time: 2.23 ms


In [148]:
def initialize_team_standings(team_data):
    teams = {}
    for index, row in team_data.iterrows():
        teams[row['Team_Name']] = {'Division': row['Division_id'], 'Conference': row['Conference_id'], 
                                   'Games_Left': 82, 'Max_Wins': 82, 'Games_Won': 0}
    return teams

time: 2.59 ms


In [149]:
def get_league_data(team_data):
    east_divisions = set(team_data[team_data['Conference_id'] == 'East']['Division_id'])
    west_divisions = set(team_data[team_data['Conference_id'] == 'West']['Division_id'])
    east_teams = set(team_data[team_data['Conference_id'] == 'East']['Team_Name'])
    west_teams = set(team_data[team_data['Conference_id'] == 'West']['Team_Name'])
    return {'East': {'Divisions': east_divisions, 'Teams': east_teams}, 
            'West': {'Divisions': west_divisions, 'Teams': west_teams}}

time: 2.85 ms


In [150]:
def initialize_league_standings(team_data):
    league_standings = {'East': {'Conference': {}, 'Atlantic': {}, 'Central': {}, 'Southeast': {}}, 
                    'West': {'Conference': {}, 'Northwest': {}, 'Southwest': {}, 'Pacific': {}}}
    for index, row in team_data.iterrows():
        league_standings[row['Conference_id']]['Conference'][row['Team_Name']] = 82
        league_standings[row['Conference_id']][row['Division_id']][row['Team_Name']] = 82
    return league_standings

time: 3.8 ms


In [482]:
day_num = 130
for team in days[day_num].league_data['East']['Teams']:
    if team not in DayOfGames.final_playoff_teams['East']:
        not_eliminated = "Unknown"
        if team in days[day_num].playoff_teams['East']:
            not_eliminated = True
            print('not')
            continue
        max_wins = days[day_num].team_standings[team]['Max_Wins']
        for other_team in days[day_num].playoff_teams['East']:
            if max_wins < days[day_num].team_standings[other_team]['Games_Won']:
                not_eliminated = False
                print('elim')
                break
        if not_eliminated != "Unknown":
            continue
        print('Unknown')
        DayOfGames.team_standings_simulation_class = copy.deepcopy(self.team_standings_simulation)
        DayOfGames.league_standings_simulation_class = copy.deepcopy(self.league_standings_simulation)

Unknown
elim
not
elim
elim
elim
Unknown
time: 7.96 ms


In [676]:
total_days = len(DayOfGames.days)
day_num = 146
team = 'New York Knicks'
if team not in DayOfGames.final_playoff_teams['East']:
    not_eliminated = "Unknown"
    if team in DayOfGames.days[day_num].playoff_teams['East']:
        not_eliminated = True
        print('not')
        #continue
    max_wins = DayOfGames.days[day_num].team_standings[team]['Max_Wins']
    if not_eliminated == "Unknown":
        DayOfGames.team_standings_simulation_class = copy.deepcopy(DayOfGames.days[day_num].team_standings_simulation)
        DayOfGames.league_standings_simulation_class = copy.deepcopy(DayOfGames.days[day_num].league_standings_simulation)
        for day in range(day_num, total_days):
            DayOfGames.days[day].simulate_day(team, True)
            DayOfGames.days[day].add_simulation_results()
            new_playoff_teams = DayOfGames.days[day].get_playoff_teams_simulation()
            if team in new_playoff_teams['East']:
                not_eliminated = True
                #print('not')
                #print(day)
                test_playoffs = DayOfGames.days[day].league_standings_simulation_class
                break
        if not_eliminated == "Unknown":
            pass
            #print('Unknown')

elim
time: 23.7 ms


In [696]:
class DayOfGames(object):
    def __init__(self, data, team_standings, league_standings):
        self.data = data
        self.date = self.data.iloc[0,0]
        self.games = self.create_game_dicts()
        self.team_standings_simulation = copy.deepcopy(DayOfGames.team_standings_class)
        self.league_standings_simulation = copy.deepcopy(DayOfGames.league_standings_class)
        self.add_game_results()
        self.team_standings = copy.deepcopy(DayOfGames.team_standings_class)
        self.league_standings = copy.deepcopy(DayOfGames.league_standings_class)
        self.playoff_teams = self.get_playoff_teams()
        
    def create_game_dicts(self):
        games = []
        for index, row in self.data.iterrows():
            game = {}
            if row['Winner'] == 'Home':
                game[row['Home Team']] = 'Winner'
                game[row['Away Team']] = 'Loser'
            else:
                game[row['Home Team']] = 'Loser'
                game[row['Away Team']] = 'Winner'
            games.append(game)
        return games    
        
    def add_game_results(self):
        for game in self.games:
            for team in game:
                self.team_standings_class[team]['Games_Left'] -= 1
                if game[team] == 'Loser':
                    self.team_standings_class[team]['Max_Wins'] -= 1
                else:
                    self.team_standings_class[team]['Games_Won'] += 1
        for team, info in self.team_standings_class.items():
            self.league_standings_class[info['Conference']]['Conference'][team] = info['Max_Wins']
            self.league_standings_class[info['Conference']][info['Division']][team] = info['Max_Wins']
            
    def simulate_day(self, team_name, win):
        self.games_simulation = copy.deepcopy(self.games)
        for game in self.games_simulation:
            if team_name in game:
                for team in game:
                    if team == team_name:
                        if win:
                            game[team] = 'Winner'
                        else:
                            game[team] = 'Loser'
                    else:
                        if win:
                            game[team] = 'Loser'
                        else:
                            game[team] = 'Winner'        
            
    def add_simulation_results(self):
        for game in self.games_simulation:
            for team in game:
                DayOfGames.team_standings_simulation_class[team]['Games_Left'] -= 1
                if game[team] == 'Loser':
                    DayOfGames.team_standings_simulation_class[team]['Max_Wins'] -= 1
                else:
                    DayOfGames.team_standings_simulation_class[team]['Games_Won'] += 1
        for team, info in DayOfGames.team_standings_simulation_class.items():
            DayOfGames.league_standings_simulation_class[info['Conference']]['Conference'][team] = info['Max_Wins']
            DayOfGames.league_standings_simulation_class[info['Conference']][info['Division']][team] = info['Max_Wins']
            
    def get_playoff_teams(self, final_day=False):
        playoff_teams_all = {}
        for conference in ['East', 'West']:
            division_leaders = []
            for division in self.league_data[conference]['Divisions']:
                team = max(self.league_standings[conference][division], 
                           key=self.league_standings[conference][division].get)
                division_leaders.append(team)
            for team, max_wins in self.league_standings[conference]['Conference'].items():
                standings = sorted(self.league_standings[conference]['Conference'], 
                                   key=self.league_standings[conference]['Conference'].get, reverse=True)
                playoff_teams = [x for x in standings if x not in division_leaders][0:5] + division_leaders
            playoff_teams_all[conference] = playoff_teams
        if final_day:
            DayOfGames.final_playoff_teams = playoff_teams_all
        else:
            return playoff_teams_all
        
    def get_playoff_teams_simulation(self):
        playoff_teams_all = {}
        for conference in ['East', 'West']:
            division_leaders = []
            for division in self.league_data[conference]['Divisions']:
                team = max(DayOfGames.league_standings_simulation_class[conference][division], 
                           key=DayOfGames.league_standings_simulation_class[conference][division].get)
                division_leaders.append(team)
            for team, max_wins in DayOfGames.league_standings_simulation_class[conference]['Conference'].items():
                standings = sorted(DayOfGames.league_standings_simulation_class[conference]['Conference'], 
                                   key=DayOfGames.league_standings_simulation_class[conference]['Conference'].get, reverse=True)
                playoff_teams = [x for x in standings if x not in division_leaders][0:5] + division_leaders
            playoff_teams_all[conference] = playoff_teams
        return playoff_teams_all
    
    @staticmethod
    def initialize_class_variables():
        DayOfGames.team_standings_class = initialize_team_standings(team_data)
        DayOfGames.team_standings_simulation_class = initialize_team_standings(team_data)
        DayOfGames.league_standings_class = initialize_league_standings(team_data)
        DayOfGames.league_standings_simulation_class = initialize_league_standings(team_data)
        DayOfGames.league_data = get_league_data(team_data)
        DayOfGames.days = []
        
    @staticmethod
    def calculate_first_day_to_check():
        elimination_days = {'East': {}, 'West': {}}
        total_days = len(DayOfGames.days)
        for conference in ['East', 'West']:
            for team in DayOfGames.league_data[conference]['Teams']:
                if team not in DayOfGames.final_playoff_teams[conference]:
                    day_num = total_days
                    eliminated = True
                    while eliminated:
                        day_num -= 1
                        max_wins = DayOfGames.days[day_num].team_standings[team]['Max_Wins']
                        can_overtake = False
                        for other_team in DayOfGames.days[day_num].playoff_teams[conference]:
                            if max_wins > DayOfGames.days[day_num].team_standings[other_team]['Games_Won']:
                                can_overtake = True
                                eliminated = False
                    elimination_days[conference][team] = day_num
        DayOfGames.first_potential_not_eliminated_day = elimination_days

time: 175 ms


In [697]:
league_data = get_league_data(team_data)
team_standings = initialize_team_standings(team_data)
league_standings = initialize_league_standings(team_data)
first_game_date = game_data.iloc[0,0]
first_game_index = 0
DayOfGames.initialize_class_variables()
for index, row in game_data.iloc[0:].iterrows():
    if row['Date'] != first_game_date:
        DayOfGames.days.append(DayOfGames(game_data.iloc[first_game_index:index], team_standings, league_standings))
        first_game_index = index
        first_game_date = row['Date']
DayOfGames.days.append(DayOfGames(game_data.iloc[first_game_index:index+1], team_standings, league_standings,))
for idx, day in enumerate(days):
    setattr(day, 'day', idx)
DayOfGames.days[-1].get_playoff_teams(True)
DayOfGames.calculate_first_day_to_check()

time: 467 ms
